#### This notebook preprocesses images for prediction using a DNN trained with Uni-EM.
It opens a series of .tiff files in `path_input` and applies CLAHE (Contrast limited adaptive histogram equalization), which enhances the local contrast of images. It then re-saves them as RGB .png files to `path_results`. After using this notebook on your raw data, the images can be used for segmentation, e.g. with Uni-EM.


In [ ]:
%matplotlib notebook
import tifffile as tf
import cv2
import matplotlib.pyplot as plt
from os import listdir
from os.path import isfile, join
import skimage
from skimage.transform import downscale_local_mean
import tqdm

In [ ]:
# user inputs
path_input = r"G:/AG_Morawski/Philip/EM/cc_test_all/test_img\\"
path_results = r"G:/AG_Morawski/Philip/EM/cc_test_all/test_img_preprocessed\\"
downscale_factor = 1 # factor 4 this is the correct factor for 
path_images = [f for f in listdir(path_input) if isfile(join(path_input, f))]

In [ ]:
print("Preprocessing Images..")

for i in tqdm.tqdm(range(len(path_images))):
    if path_images[i].endswith(".tif") or path_images[i].endswith(".tiff"):
        test_img = tf.imread(path_input + str(path_images[i])) # use this for tiff
    if path_images[i].endswith(".png"):
        test_img = cv2.imread(path_input + str(path_images[i]),-1)[:,:,0] # use this for png 
    else:
        print('Input file format not supported. Use .png or .tif.')
        break
    test_img_clahe = skimage.exposure.equalize_adapthist(test_img,clip_limit=0.01,kernel_size=127)
    test_img_downscaled = downscale_local_mean(test_img_clahe, downscale_factor)
    test_img_rgb_png = cv2.merge((downscale_local_mean(test_img_downscaled,1),  #R
                                downscale_local_mean(test_img_downscaled,1),    #G
                                downscale_local_mean(test_img_downscaled,1)))   #B
    skimage.io.imsave(path_results + path_images[i][:-4] + ".png", (test_img_rgb_png*255).astype('uint8'))
